In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
from scipy.stats import entropy

In [3]:
all_keypoint_metrics = pd.read_csv("Data/all_keypoint_metrics.csv")

ori_AU_consistency = np.loadtxt("Data/AU_consistency.txt") 

NameError: name 'pd' is not defined

In [4]:
avg_t = np.array(all_keypoint_metrics['Avg_t'])
max_t = np.array(all_keypoint_metrics['Max_t'])
avg_t2 = np.array(all_keypoint_metrics['Avg_Bivariate_HT2'])
max_t2 = np.array(all_keypoint_metrics['Max_Bivariate_HT2'])
HT_t2 = np.array(all_keypoint_metrics['Hotelling_t_2_PCA'])



NameError: name 'all_keypoint_metrics' is not defined

### Computing all the regression plots

In [ ]:
AU_consistency = 100*ori_AU_consistency/27
df_temp = pd.DataFrame(AU_consistency, columns=['AU_consistency'])
df_temp = pd.concat([df_temp, all_keypoint_metrics], axis=1)
  
metric_dict = {'Avg_t':"Average $\it{t}$-statistic", 'Max_t':"Maximum $\it{t}$-statistic", 'Avg_t2':"Average $\it{t}^{2}$-statistic", 'Max_t2':"Maximum $\it{t}^{2}$-statistic", 't2_with_PCA':"$t^{2}$-statistic with PCA"}

for metric in metric_dict.keys():
    fig, ax = plt.subplots()
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    
    # all the plt features will work if placed after lmplot function
    sb.lmplot(x = metric,
                y = "AU_consistency", 
                ci = None,
                data = df_temp, height=10, aspect=1, markers=".", line_kws={'color': 'purple'})
    plt.ylim(0,110)
#     plt.xlim(0,5.5)
    plt.xlabel(metric_dict[metric], fontsize=28, fontweight='bold')
    plt.ylabel("AU Consistency", fontsize=28, fontweight='bold')
#     plt.rcParams["axes.edgecolor"] = "black"
#     plt.rcParams["figure.autolayout"] = True
    plt.gca().spines['top'].set_visible(True)
    plt.gca().spines['right'].set_visible(True)
    plt.savefig("Results/"+metric+".jpg", dpi=300)
    plt.close()

### comparing Avg t-statistic and AU consistency along the video timeline

In [ ]:
# y = reg.predict(avg_disfa_t.reshape(-1,1))
y = AU_consistency.reshape(-1,1)
y = y/np.amax(y)
print(y.shape)
# y = avg_selfdisfa3741_t.reshape(-1,1)
x = np.arange(0, y.shape[0], 1)
plt.figure(figsize=(20,10))
mx=1.1
plt.ylim(0,mx)
# fig, axes = plt.subplot(1,1)
for key in emotions_dict:
    color = emotions_dict[key][0]
    emotion = key
    a, b = emotions_dict[key][1][0], emotions_dict[key][1][1] 
    plt.fill_between(x, 0, mx*np.ones(y.shape[0]), np.multiply(x>=a,x<b), color=color, label=emotion)
# legend1 = plt.legend(loc='upper right', prop={'weight':'bold','size':14})
legend1 = plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), prop={'weight':'bold', 'size':39.5})
# plt.plot(x, y, color='blue', label='DISFA')
# plt.legend(loc='upper left')
# y2 = reg.predict(avg_selfdisfa_t.reshape(-1,1))
y2 = avg_t.reshape(-1,1)
y2 = y2/np.amax(y2)
plt.plot(x, y, color='navy', label='AU consistency')
plt.plot(x, y2, color='purple', label='Average t-statistic')
# plt.plot(x, y, color='tab:orange', label='AU consistency')
# plt.plot(x, y2, color='tab:green', label='Average t-statistic')
plt.legend(['AU consistency','Average t-statistic'],loc='upper right', prop={'weight':'bold','size':20})
# Uncomment the below line to add the emotion color labels on side
plt.gca().add_artist(legend1)
plt.xlabel("Video Timeline (in Frames)", fontsize=20, fontweight='bold')
plt.ylabel("Consistency", fontsize=20, fontweight='bold')
# plt.vlines([191,771,1718], 0, [mx,mx,mx], linestyle="dashed", color='black')
# plt.xticks(np.concatenate((np.arange(0,5000,1000),np.array([191, 771, 1718]))))
y_intercept=2.04
# plt.axhline(y=y_intercept, linestyle='--', color='gray')
# plt.text(0.1, y_intercept+0.07, f'y={y_intercept}', ha='right', va='center')
# plt.show()
plt.savefig("Figures_for_paper/disfa_compare_metrics.pdf", dpi=300, bbox_inches='tight')
plt.close()

### Creating class distribution of all the metrics

In [ ]:
# Emotions dictionary
emotions_dict = {
    "Happy0": ['thistle', [64, 341]],
    "Happy1": ['lavender', [341, 816]],
    "Surprise0": ['mediumpurple', [816, 1232]],
    "Fear": ['slateblue', [1232, 1413]],
    "Disgust0": ['lightgrey', [1413, 1934]],
    "Disgust1": ['royalblue', [1934, 2514]],
    "Sadness0": ['deepskyblue', [2514, 3264]],
    "Sadness1": ['lightsteelblue', [3264, 3993]],
    "Surprise1": ['cornflowerblue', [3993, 4831]]
}

In [ ]:
import numpy as np
import pandas as pd

percentages_list = []
for x in [AU_consistency, avg_t, max_t, avg_t2, max_t2, HT_t2]:

    # Define quartiles function
    def calculate_quartiles(arr):
        q1 = np.percentile(arr, 30)
        q2 = np.percentile(arr, 60)
        q3 = np.percentile(arr, 90)
        return q1, q2, q3

    # Create a DataFrame to store the results
    result_df = pd.DataFrame(columns=["Inconsistent", "Low Consistent", "Mild Consistent", "High Consistent"])

    # Iterate through each emotion in the dictionary
    for emotion, (color, index_range) in emotions_dict.items():
        start_index, end_index = index_range

        # Extract values within the specified index range
        subset = x[start_index:end_index]

        # Calculate quartiles
        q1, q2, q3 = calculate_quartiles(x)
        print(q1, q2, q3, np.amax(x))

        # Categorize values
        category = np.zeros_like(subset, dtype=int)
        category[subset < q1] = 1
        category[(subset >= q1) & (subset < q2)] = 2
        category[(subset >= q2) & (subset < q3)] = 3
        category[subset >= q3] = 4

        # Calculate percentages
        percentages = np.array([int(np.round(np.mean(category == i) * 100)) for i in range(1, 5)])

        # Add row to the DataFrame
        result_df.loc[emotion] = percentages
        # Add mean, max, and min for each column
    percentages_list.append(result_df.to_numpy(dtype=np.float))
    # Display the result
#     result_df.loc['Mean'] = result_df.mean()
#     result_df.loc['Max'] = result_df.max()
#     result_df.loc['Min'] = result_df.min()
    print(result_df)


### Computing KL-divergence between the tables

In [ ]:
# Ensure that all elements in percentages_list are NumPy arrays
for array in percentages_list:
    if not isinstance(array, np.ndarray):
        raise ValueError("All elements in percentages_list should be NumPy arrays.")

# Extract the first array
first_array = percentages_list[0]

# Extract Category rows from the first array
category_rows_first_array = [first_array[i, :] for i in range(9)]

# Extract the last five arrays
last_five_arrays = percentages_list[1:]

# Initialize a 5x9 matrix to store KL divergences
kl_divergences_matrix = np.zeros((5, 9))

# Calculate KL divergence for each combination of Category rows
for i, category_row_first_array in enumerate(category_rows_first_array):
    for j, array in enumerate(last_five_arrays):
        category_row_array = array[i, :]
        
        # Add a small epsilon to avoid division by zero
        epsilon = 1e-10
        kl_divergence = entropy(category_row_first_array/100 + epsilon, category_row_array/100 + epsilon)
#         kl_divergence = entropy(category_row_array/100 + epsilon, category_row_first_array/100 + epsilon)
        kl_divergences_matrix[j, i] = kl_divergence

# Display the KL Divergences matrix
print("KL Divergences Matrix:")
print(kl_divergences_matrix)

# Compute the average of each row
average_divergences_per_row = np.mean(kl_divergences_matrix, axis=1)

# Display the average divergences
print("\nAverage KL Divergences per Row:")
print(average_divergences_per_row)
